In [59]:
import os
import re
import spacy
import hashlib
import numpy as np
import pandas as pd

from tqdm import tqdm
from collections import Counter
from spacy.matcher import Matcher
from sklearn.metrics import accuracy_score
from nltk.tokenize import WordPunctTokenizer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer

In [60]:
df_dataset = pd.read_csv("data\dataset_mapped.csv", index_col=0)
df_dataset.head()

,text,date,category,language,class
0,"Hello, Does it matter iff I use Visa or Master...",7-6-2022,visa_or_mastercard,en,card
1,"Good afternoon, I just got refunded for my pur...",16-11-2022,reverted_card_payment?,en,card
2,"Hello, I got billed ann extra pound! Thanks",4-12-2022,extra_charge_on_statement,en,others
3,"Hi, How long does it take for a transfer to sh...",23-11-2022,transfer_timing,en,transfer
4,"hi, When can I use money sent to my accountt? ...",17-4-2022,transfer_timing,en,transfer


In [61]:
df_dataset.dtypes

text        object
date        object
category    object
language    object
class       object
dtype: object

In [62]:
df_dataset = df_dataset.loc[df_dataset['language'] == 'en']
df_dataset.head()

#df.drop(df.loc[df['line_race']==0].index, inplace=True)

,text,date,category,language,class
0,"Hello, Does it matter iff I use Visa or Master...",7-6-2022,visa_or_mastercard,en,card
1,"Good afternoon, I just got refunded for my pur...",16-11-2022,reverted_card_payment?,en,card
2,"Hello, I got billed ann extra pound! Thanks",4-12-2022,extra_charge_on_statement,en,others
3,"Hi, How long does it take for a transfer to sh...",23-11-2022,transfer_timing,en,transfer
4,"hi, When can I use money sent to my accountt? ...",17-4-2022,transfer_timing,en,transfer


In [63]:
df_dataset["text"] = df_dataset["text"].astype(str)
#df_dataset["category"] = df_dataset["category"].astype(str)
#df_dataset["language"] = df_dataset["language"].astype(str)
df_dataset["class"] = df_dataset["class"].astype(str)

In [64]:
df_dataset.dtypes

text        object
date        object
category    object
language    object
class       object
dtype: object

In [65]:
df_dataset.drop(columns=["category", "language", "date"], inplace=True)
df_dataset.head()

,text,class
0,"Hello, Does it matter iff I use Visa or Master...",card
1,"Good afternoon, I just got refunded for my pur...",card
2,"Hello, I got billed ann extra pound! Thanks",others
3,"Hi, How long does it take for a transfer to sh...",transfer
4,"hi, When can I use money sent to my accountt? ...",transfer


In [66]:

def remove_punctuation(text):
    """
    Hint: Remember the good old RegEx from 2 LUs ago
        how can I just remove everything except words, digits and spaces?
    """
    
    text = re.sub(r'[^\w\s]+', '', text)

    return text.lower()



In [69]:
def preprocess_text(df):
    
    df_processed = df.copy()
    df_processed["text"] = df_processed["text"].apply(lambda x: remove_punctuation(x))
    return df_processed
df_processed = preprocess_text(df_dataset)


In [70]:
df_processed.head()

,text,class
0,hello does it matter iff i use visa or masterc...,card
1,good afternoon i just got refunded for my purc...,card
2,hello i got billed ann extra pound thanks,others
3,hi how long does it take for a transfer to sho...,transfer
4,hi when can i use money sent to my accountt be...,transfer
